In [1]:
!pip install max --index-url https://dl.modular.com/public/nightly/python/simple/

Looking in indexes: https://dl.modular.com/public/nightly/python/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 285.0/285.0 MB 5.4 MB/s eta 0:00:00


In [2]:
import max.support.notebook

In [3]:
!git clone https://github.com/modular/mojo-gpu-puzzles

Cloning into 'mojo-gpu-puzzles'...
remote: Enumerating objects: 4953, done.
remote: Counting objects: 100% (384/384), done.
remote: Compressing objects: 100% (67/67), done.
remote: Total 4953 (delta 339), reused 329 (delta 312), pack-reused 4569 (from 1)
Receiving objects: 100% (4953/4953), 98.42 MiB | 23.04 MiB/s, done.
Resolving deltas: 100% (3078/3078), done.


In [4]:
%cd mojo-gpu-puzzles

/content/mojo-gpu-puzzles


In [5]:
!curl -fsSL https://astral.sh/uv/install.sh | sh

downloading uv 0.7.16 x86_64-unknown-linux-gnu
no checksums to verify
installing to /usr/local/bin
  uv
  uvx
everything's installed!


In [6]:
mojo_code = """
from memory import UnsafePointer
from gpu import thread_idx
from gpu.host import DeviceContext
from testing import assert_equal

# ANCHOR: add_10
alias SIZE = 4
alias BLOCKS_PER_GRID = 1
alias THREADS_PER_BLOCK = SIZE
alias dtype = DType.float32


fn add_10(output: UnsafePointer[Scalar[dtype]], a: UnsafePointer[Scalar[dtype]]):
    i = thread_idx.x
    # FILL ME IN (roughly 1 line)
    output[i] = a[i] + 10


# ANCHOR_END: add_10


def main():
    with DeviceContext() as ctx:
        out = ctx.enqueue_create_buffer[dtype](SIZE)
        out = out.enqueue_fill(0)
        a = ctx.enqueue_create_buffer[dtype](SIZE)
        a = a.enqueue_fill(0)
        with a.map_to_host() as a_host:
            for i in range(SIZE):
                a_host[i] = i

        ctx.enqueue_function[add_10](
            out.unsafe_ptr(),
            a.unsafe_ptr(),
            grid_dim=BLOCKS_PER_GRID,
            block_dim=THREADS_PER_BLOCK,
        )

        expected = ctx.enqueue_create_host_buffer[dtype](SIZE)
        expected = expected.enqueue_fill(0)
        ctx.synchronize()

        for i in range(SIZE):
            expected[i] = i + 10

        with out.map_to_host() as out_host:
            print("out:", out_host)
            print("expected:", expected)
            for i in range(SIZE):
                assert_equal(out_host[i], expected[i])
                """

In [7]:
def save_code_to_file(text: str, filename: str):
    with open(filename, 'w', encoding='utf-8') as file:
        file.write(text)
save_code_to_file(mojo_code, "/content/mojo-gpu-puzzles/problems/p01/p01.mojo")

In [8]:
!cd /content/mojo-gpu-puzzles && uv run poe p01

Streaming output truncated to the last 5000 lines.
nvidia-cusparselt-cu12   ------------------------------ 110.00 MiB/149.52 MiB
nvidia-cufft-cu12        ------------------------------ 112.82 MiB/184.17 MiB
nvidia-nccl-cu12         ------------------------------ 110.53 MiB/191.99 MiB
nvidia-cusolver-cu12     ------------------------------ 111.53 MiB/248.31 MiB
nvidia-cusparse-cu12     ------------------------------ 111.31 MiB/278.53 MiB
nvidia-cublas-cu12       ------------------------------ 111.69 MiB/581.38 MiB
nvidia-cudnn-cu12        ------------------------------ 111.93 MiB/693.18 MiB
⠧ Preparing packages... (21/30)
triton                   ------------------------------ 105.00 MiB/148.48 MiB
nvidia-cusparselt-cu12   ------------------------------ 110.50 MiB/149.52 MiB
nvidia-cufft-cu12        ------------------------------ 113.09 MiB/184.17 MiB
nvidia-nccl-cu12         ------------------------------ 110.53 MiB/191.99 MiB
nvidia-cusolver-cu12     ------------------------------ 111